In [1]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [2]:
#载入数据集
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#运行次数
max_steps = 1001
#图片数量
image_num = 3000
#文件路径
DIR = "D:/Tensorflow/"
#载入图片
embedding = tf.Variable(tf.stack(mnist.test.images[:image_num]),trainable=False,name='embedding')

#参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) #平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev) #标准差
        tf.summary.scalar('max',tf.reduce_max(var)) #最大值
        tf.summary.scalar('min',tf.reduce_min(var)) #最小值
        tf.summary.histogram('histogram',var) #直方图

#命名空间
with tf.name_scope('input'):
    #这里的none表示第一个维度可以是任意的长度
    x = tf.placeholder(tf.float32,[None,784],name='x-input')
    #正确的标签
    y = tf.placeholder(tf.float32,[None,10],name='y-input')
    
#显示图片
with tf.name_scope('input_reshape'):
    image_shaped_input = tf.reshape(x,[-1,28,28,1])
    tf.summary.image('input',image_shaped_input,10) #显示10张图片

with tf.name_scope('layer'):
    #创建一个简单的神经网络
    with tf.name_scope('wights'):
        W = tf.Variable(tf.zeros([784,10]),name='W')
        variable_summaries(W) #分析权重的变化
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]),name='b')
        variable_summaries(b) #分析偏置值的变化
    with tf.name_scope('wx_plus_b'):
        wx_plus_b = tf.matmul(x,W)+b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(wx_plus_b)
    
with tf.name_scope('loss'):
    #交叉熵代价函数 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('loss',loss)
with tf.name_scope('train'):
    #使用梯度下降法进行训练
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        #结果存放在一个布尔型列表中
        correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(prediction,1)) #argmax返回一维张量中最大值所在的位置
    with tf.name_scope('accuracy'):
        #求准确率
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #cast表示类型转换
        tf.summary.scalar('accuracy',accuracy)

#合并所有的summary
merged = tf.summary.merge_all()
#初始化变量
init = tf.global_variables_initializer() 
#启动会话
with tf.Session() as sess:
    sess.run(init)
    #产生metadata文件
    if tf.gfile.Exists(DIR + 'projector/projector/metadata.tsv'):
        tf.gfile.DeleteRecursively(DIR + 'projector/projector/metadata.tsv')
    with open(DIR + 'projector/projector/metadata.tsv','w') as f:
        labels = sess.run(tf.argmax(mnist.test.labels[:],1)) #argmax()最大值所在的位置
        for i in range(image_num):
            f.write(str(labels[i]) + '\n')
        
    projector_writer = tf.summary.FileWriter(DIR + 'projector/projector',sess.graph) #计算图保存在指定路径中
    saver = tf.train.Saver() #保存网络的模型
    #建立 embedding projector（指定想要可视化的 variable，metadata 文件的位置）
    config = projector.ProjectorConfig() #定义配置项
    embed = config.embeddings.add()
    embed.tensor_name = embedding.name
    embed.metadata_path = DIR + 'projector/projector/metadata.tsv'
    embed.sprite.image_path = DIR + 'projector/data/mnist_10k_sprite.png' #图片切分
    embed.sprite.single_image_dim.extend([28,28]) #单个图片28x28
    projector.visualize_embeddings(projector_writer,config)

    for i in range(max_steps):
        #每个批次100个样本
        batch_xs,batch_ys = mnist.train.next_batch(100)
        run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE) #配置运行时需要记录的信息
        run_metadata = tf.RunMetadata()  #运行时记录运行信息的proto
        #将配置信息和记录运行信息的proto传入运行的过程，从而记录运行时每一个节点的时间、空间开销信息
        summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys},options=run_options,run_metadata=run_metadata)
        projector_writer.add_run_metadata(run_metadata, 'step%03d' % i) #将节点在运行时的信息写入日志文件
        projector_writer.add_summary(summary, i)
    
        if i%100 == 0:
            acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print("Iter " + str(i) + ", Testing Accuracy= " + str(acc))

    saver.save(sess,DIR + 'projector/projector/a_model.ckpt',global_step=max_steps)
    projector_writer.close()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Iter 0, Testing Accuracy= 0.2108
Iter 100, Testing Accuracy= 0.8022
Iter 200, Testing Accuracy= 0.8191
Iter 300, Testing Accuracy= 0.8273
Iter 400, Testing Accuracy= 0.8755
Iter 500, Testing Accuracy= 0.8877
Iter 600, Testing Accuracy= 0.8958
Iter 700, Testing Accuracy= 0.8999
Iter 800, Testing Accuracy= 0.9036
Iter 900, Testing Accuracy= 0.905
Iter 1000, Testing Accuracy= 0.906
